# Introduction: Results of Time Feature Engineering

In this notebook, we'll look at the results of the time feature engineering. The purpose is to determine (if possible) the best way for encoding time and date features in a time-series problem.

In [1]:
# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.set_config_file(world_readable=True, theme="pearl")
cf.go_offline(connected=True)

# Extra options
pd.options.display.max_rows = 10
pd.options.display.max_columns = 25
# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [30]:
lr = pd.read_csv('results/linear_model.csv', index_col=0).reset_index(drop=True)
rr= pd.read_csv('results/random_forest_model.csv', index_col=0).reset_index(drop=True)

lr = lr.loc[(np.isfinite(lr['score'])) & (lr['score'] > 0)]
rr = rr.loc[(np.isfinite(rr['score'])) & (rr['score'] > 0)]

In [31]:
lr.describe()

,score,freq,dpoints,building_id
count,1050.000000,1050.000000,1050.000000,1050.000000
mean,0.594523,53.314286,17787.822857,346.567619
std,0.244286,16.012575,23293.782530,173.970817
min,0.004943,15.000000,8760.000000,2.000000
25%,0.440051,60.000000,8760.000000,238.000000
50%,0.643491,60.000000,8760.000000,375.000000
75%,0.788277,60.000000,8784.000000,493.000000
max,0.976023,60.000000,139716.000000,600.000000


In [32]:
rr.describe()

,score,freq,dpoints,building_id
count,1877.000000,1877.000000,1877.000000,1877.000000
mean,0.761161,53.407032,17381.578050,337.102291
std,0.169312,15.917013,22380.627153,171.355788
min,0.027460,15.000000,8760.000000,1.000000
25%,0.698803,60.000000,8760.000000,225.000000
50%,0.802353,60.000000,8760.000000,358.000000
75%,0.877204,60.000000,8784.000000,481.000000
max,0.983892,60.000000,139716.000000,600.000000


In [35]:
rr.pivot_table(index='building_id', values='score', columns='method')

method,frac,frac_cyc,normal,normal_cyc
building_id,,,,
1,0.712770,0.686601,0.691134,0.279083
2,0.817534,0.827222,0.819969,0.802353
5,0.806778,0.815835,0.794974,0.807480
6,0.691102,0.696016,0.674050,0.497206
7,0.472874,0.553007,0.516633,0.400233
...,...,...,...,...
596,0.757680,0.747749,0.787795,0.675157
597,0.886680,0.909640,0.880711,0.868342
598,0.886729,0.909629,0.880679,0.868175


In [36]:
rr.pivot_table(index='building_id', values='score', columns='method').iplot(mode='markers', size=6)

In [37]:
rr.groupby('building_id').apply(lambda x: x.loc[x['score'].idxmax(), 'method']).value_counts()

frac_cyc      247
normal        128
normal_cyc     55
frac           41
dtype: int64

In [38]:
rr.groupby('method')['score'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
frac,469.0,0.755411,0.167938,0.083785,0.691102,0.797031,0.872455,0.982494
frac_cyc,469.0,0.774670,0.165859,0.037346,0.719130,0.815217,0.886487,0.981503
normal,471.0,0.764776,0.163083,0.032756,0.711770,0.800524,0.877222,0.983892
normal_cyc,468.0,0.749748,0.179438,0.027460,0.676513,0.797247,0.872617,0.982997


# Fractional Cyclical Encoding Works Best

In [39]:
rr['score'].idxmax()

304

In [40]:
rr.loc[rr['score'].idxmax()]

method           normal
score          0.983892
freq                 60
dpoints            8760
building_id         538
Name: 304, dtype: object

In [45]:
rr.pivot_table(index='building_id', columns='method', values='score').iplot(kind='box')

In [44]:
rr.pivot_table(index='building_id', columns='method', values='score')

method,frac,frac_cyc,normal,normal_cyc
building_id,,,,
1,0.712770,0.686601,0.691134,0.279083
2,0.817534,0.827222,0.819969,0.802353
5,0.806778,0.815835,0.794974,0.807480
6,0.691102,0.696016,0.674050,0.497206
7,0.472874,0.553007,0.516633,0.400233
...,...,...,...,...
596,0.757680,0.747749,0.787795,0.675157
597,0.886680,0.909640,0.880711,0.868342
598,0.886729,0.909629,0.880679,0.868175
